In [10]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트
%matplotlib inline

In [11]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
dog_train=pd.read_csv("testDog.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)


In [12]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)
print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [13]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)


# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)


In [14]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [15]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)


       weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  \
0        7.46          10    12             0    0    0    0    0    0    0   
1        7.00          14     1             1    0    0    0    0    0    0   
2        4.50          11     2             0    0    0    0    0    0    0   
3       10.00           8     1             0    0    0    0    0    0    0   
4        6.00           8     4             0    0    0    0    0    0    0   
...       ...         ...   ...           ...  ...  ...  ...  ...  ...  ...   
22787    1.00          10     0             1    0    0    0    0    0    0   
22788    1.00          10     0             1    0    0    0    0    0    0   
22789    1.00          10     0             1    0    0    0    0    0    0   
22790    6.00          12     3             0    0    0    0    0    0    0   
22791    3.50          10     0             0    0    0    0    0    0    0   

       ...  208.0  209.0  210.0  211.0  neuterYn_N 

In [24]:
a = data[:, :3]
b = data[:, 4:]

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a=train_data[:, :3]
b=train_data[:, 4:]
testX=np.concatenate([a, b], axis = 1)
testX.shape

(610, 186)

In [17]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
testY=train_data[:,[3]]

print(yData)
type(yData)


[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [25]:
# L2규제만
l2_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.01),
                       activation=tf.nn.relu, input_shape=(186,)),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.01),
                       activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
sgd=optimizers.SGD(lr=0.01)

l2_model.compile(optimizer='sgd',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

l2_model_history = l2_model.fit(xData, yData,
                                epochs=100,
                                batch_size=32,
                                validation_data=(testX, testY),
                                verbose=1)

l2_model.summary()
loss_and_metric = l2_model.evaluate(testX, testY, batch_size=32, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))

Train on 22777 samples, validate on 610 samples
Epoch 1/100
22777/22777 [==============================] - 3s 138us/sample - loss: 1.0623 - acc: 0.6136 - val_loss: 1.0188 - val_acc: 0.5852
Epoch 2/100
22777/22777 [==============================] - 2s 88us/sample - loss: 0.9487 - acc: 0.6384 - val_loss: 0.9315 - val_acc: 0.6246
Epoch 3/100
22777/22777 [==============================] - 2s 101us/sample - loss: 0.8675 - acc: 0.6547 - val_loss: 0.8670 - val_acc: 0.6295
Epoch 4/100
22777/22777 [==============================] - 2s 109us/sample - loss: 0.8101 - acc: 0.6562 - val_loss: 0.8093 - val_acc: 0.6328
Epoch 5/100
22777/22777 [==============================] - 2s 76us/sample - loss: 0.7667 - acc: 0.6570 - val_loss: 0.7629 - val_acc: 0.6246
Epoch 6/100
22777/22777 [==============================] - 2s 97us/sample - loss: 0.7344 - acc: 0.6569 - val_loss: 0.7415 - val_acc: 0.6262
Epoch 7/100
22777/22777 [==============================] - 2s 99us/sample - loss: 0.7103 - acc: 0.6597 - val_

22777/22777 [==============================] - 2s 101us/sample - loss: 0.6266 - acc: 0.6681 - val_loss: 0.6521 - val_acc: 0.6475
Epoch 59/100
22777/22777 [==============================] - 2s 83us/sample - loss: 0.6295 - acc: 0.6682 - val_loss: 0.6675 - val_acc: 0.6279
Epoch 60/100
22777/22777 [==============================] - 2s 67us/sample - loss: 0.6270 - acc: 0.6661 - val_loss: 0.6463 - val_acc: 0.6410
Epoch 61/100
22777/22777 [==============================] - 2s 105us/sample - loss: 0.6287 - acc: 0.6669 - val_loss: 0.6480 - val_acc: 0.6311
Epoch 62/100
22777/22777 [==============================] - 2s 85us/sample - loss: 0.6260 - acc: 0.6701 - val_loss: 0.6655 - val_acc: 0.6279
Epoch 63/100
22777/22777 [==============================] - 1s 64us/sample - loss: 0.6278 - acc: 0.6669 - val_loss: 0.6520 - val_acc: 0.6492
Epoch 64/100
22777/22777 [==============================] - 2s 83us/sample - loss: 0.6267 - acc: 0.6698 - val_loss: 0.6469 - val_acc: 0.6475
Epoch 65/100
22777/22777

In [26]:
# 드롭아웃만

dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(186,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

sgd=optimizers.SGD(lr=0.01)

# l2_model.compile(optimizer='sgd',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy', 'binary_crossentropy'])


dpt_model.compile(optimizer='sgd',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

dpt_model_history = dpt_model.fit(xData, yData,
                                  epochs=100,
                                  batch_size=32,
                                  validation_data=(testX, testY),
                                  verbose=1)

loss_and_metric = dpt_model.evaluate(testX, testY, batch_size=32, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))

Train on 22777 samples, validate on 610 samples
Epoch 1/100
22777/22777 [==============================] - 3s 119us/sample - loss: 0.6800 - acc: 0.5729 - val_loss: 0.6773 - val_acc: 0.5557
Epoch 2/100
22777/22777 [==============================] - 3s 120us/sample - loss: 0.6639 - acc: 0.6052 - val_loss: 0.6669 - val_acc: 0.5852
Epoch 3/100
22777/22777 [==============================] - 2s 98us/sample - loss: 0.6534 - acc: 0.6251 - val_loss: 0.6565 - val_acc: 0.6131
Epoch 4/100
22777/22777 [==============================] - 2s 82us/sample - loss: 0.6440 - acc: 0.6384 - val_loss: 0.6507 - val_acc: 0.6180
Epoch 5/100
22777/22777 [==============================] - 2s 102us/sample - loss: 0.6383 - acc: 0.6458 - val_loss: 0.6447 - val_acc: 0.6361
Epoch 6/100
22777/22777 [==============================] - 3s 127us/sample - loss: 0.6315 - acc: 0.6492 - val_loss: 0.6398 - val_acc: 0.6311
Epoch 7/100
22777/22777 [==============================] - 2s 100us/sample - loss: 0.6277 - acc: 0.6564 - va

22777/22777 [==============================] - 2s 105us/sample - loss: 0.6067 - acc: 0.6745 - val_loss: 0.6365 - val_acc: 0.6426
Epoch 59/100
22777/22777 [==============================] - 2s 90us/sample - loss: 0.6067 - acc: 0.6699 - val_loss: 0.6373 - val_acc: 0.6525
Epoch 60/100
22777/22777 [==============================] - 2s 104us/sample - loss: 0.6073 - acc: 0.6717 - val_loss: 0.6310 - val_acc: 0.6475
Epoch 61/100
22777/22777 [==============================] - 2s 107us/sample - loss: 0.6054 - acc: 0.6728 - val_loss: 0.6395 - val_acc: 0.6459
Epoch 62/100
22777/22777 [==============================] - 2s 108us/sample - loss: 0.6059 - acc: 0.6728 - val_loss: 0.6328 - val_acc: 0.6443
Epoch 63/100
22777/22777 [==============================] - 2s 109us/sample - loss: 0.6075 - acc: 0.6729 - val_loss: 0.6423 - val_acc: 0.6393
Epoch 64/100
22777/22777 [==============================] - 2s 104us/sample - loss: 0.6065 - acc: 0.6702 - val_loss: 0.6326 - val_acc: 0.6459
Epoch 65/100
22777/2

In [22]:
# L2규제, 드롭아웃 모두 적용

l2_dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.01),
                       activation=tf.nn.relu, input_shape=(186,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.01),
                       activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
]) 
adam=optimizers.Adam(lr=0.01)
sgd=optimizers.SGD(lr=0.01)
l2_dpt_model.compile(optimizer=sgd,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

l2_model.fit(xData, yData,epochs=100,batch_size=64,validation_data=(xData, yData),verbose=1)
loss_and_metric = l2_dpt_model.evaluate(testX, testY, batch_size=32, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))

Train on 22777 samples, validate on 22777 samples
Epoch 1/100
22777/22777 [==============================] - 1s 42us/sample - loss: 0.6213 - acc: 0.6727 - val_loss: 0.6217 - val_acc: 0.6682
Epoch 2/100
22777/22777 [==============================] - 1s 49us/sample - loss: 0.6221 - acc: 0.6721 - val_loss: 0.6161 - val_acc: 0.6786
Epoch 3/100
22777/22777 [==============================] - 1s 55us/sample - loss: 0.6222 - acc: 0.6705 - val_loss: 0.6197 - val_acc: 0.6772
Epoch 4/100
22777/22777 [==============================] - 1s 52us/sample - loss: 0.6222 - acc: 0.6729 - val_loss: 0.6146 - val_acc: 0.6822
Epoch 5/100
22777/22777 [==============================] - 1s 63us/sample - loss: 0.6216 - acc: 0.6710 - val_loss: 0.6190 - val_acc: 0.6712
Epoch 6/100
22777/22777 [==============================] - 1s 42us/sample - loss: 0.6211 - acc: 0.6737 - val_loss: 0.6146 - val_acc: 0.6801
Epoch 7/100
22777/22777 [==============================] - 1s 46us/sample - loss: 0.6213 - acc: 0.6705 - val_l

Epoch 59/100
22777/22777 [==============================] - 1s 43us/sample - loss: 0.6217 - acc: 0.6727 - val_loss: 0.6219 - val_acc: 0.6673
Epoch 60/100
22777/22777 [==============================] - 1s 41us/sample - loss: 0.6207 - acc: 0.6740 - val_loss: 0.6159 - val_acc: 0.6748
Epoch 61/100
22777/22777 [==============================] - 1s 52us/sample - loss: 0.6201 - acc: 0.6734 - val_loss: 0.6141 - val_acc: 0.6795
Epoch 62/100
22777/22777 [==============================] - 1s 52us/sample - loss: 0.6210 - acc: 0.6716 - val_loss: 0.6154 - val_acc: 0.6805
Epoch 63/100
22777/22777 [==============================] - 1s 39us/sample - loss: 0.6212 - acc: 0.6723 - val_loss: 0.6165 - val_acc: 0.6780
Epoch 64/100
22777/22777 [==============================] - 1s 58us/sample - loss: 0.6209 - acc: 0.6701 - val_loss: 0.6259 - val_acc: 0.6626
Epoch 65/100
22777/22777 [==============================] - 1s 41us/sample - loss: 0.6218 - acc: 0.6719 - val_loss: 0.6475 - val_acc: 0.6486
Epoch 66/100


In [15]:
for i in range(len(train_data)):
    new_x=train_data[i, :].reshape(1,186)
    print('입양 확률 : %8.4f%%' % (l2_model.predict(new_x)*100))
l2_model.save('l2_model')

입양 확률 :  94.2203%
입양 확률 :  36.7390%
입양 확률 :  35.8996%
입양 확률 :  78.0932%
입양 확률 :  21.0893%
입양 확률 :  21.0893%
입양 확률 :  91.3862%
입양 확률 :  78.4416%
입양 확률 :  89.5666%
입양 확률 :  84.8468%
입양 확률 :  72.6395%
입양 확률 :  85.3167%
입양 확률 :  55.5937%
입양 확률 :  49.5257%
입양 확률 :  51.3644%
입양 확률 :  28.9706%
입양 확률 :  44.0449%
입양 확률 :  37.9210%
입양 확률 :  83.8701%
입양 확률 :  23.1563%
입양 확률 :  64.8517%
입양 확률 :  64.8517%
입양 확률 :  53.8228%
입양 확률 :  46.9142%
입양 확률 :  71.4867%
입양 확률 :  58.0059%
입양 확률 :  54.1604%
입양 확률 :  57.8769%
입양 확률 :  41.1052%
입양 확률 :  54.8027%
입양 확률 :  56.1087%
입양 확률 :  41.1052%
입양 확률 :  40.0840%
입양 확률 :  37.2096%
입양 확률 :  90.8167%
입양 확률 :  88.6520%
입양 확률 :  19.2490%
입양 확률 :  69.2019%
입양 확률 :  76.6373%
입양 확률 :  28.4616%
입양 확률 :  26.2082%
입양 확률 :  37.8162%
입양 확률 :  49.3927%
입양 확률 :  31.0500%
입양 확률 :  32.5425%
입양 확률 :  45.5628%
입양 확률 :  49.3927%
입양 확률 :  51.3507%
입양 확률 :  36.3324%
입양 확률 :  45.9654%
입양 확률 :  29.5362%
입양 확률 :  29.1457%
입양 확률 :  55.1988%
입양 확률 :  52.2923%
입양 확률 :  54.0260%
입양 확률 :  5

입양 확률 :  28.6237%
입양 확률 :  93.3353%
입양 확률 :  32.6441%
입양 확률 :  37.7122%
입양 확률 :  67.2223%
입양 확률 :  23.0380%
입양 확률 :  70.3892%
입양 확률 :  77.7126%
입양 확률 :  55.1501%
입양 확률 :  57.7917%
입양 확률 :  67.0752%
입양 확률 :  32.6870%
입양 확률 :  88.6794%
입양 확률 :  50.1714%
입양 확률 :  34.4334%
입양 확률 :  30.2486%
입양 확률 :  57.1593%
입양 확률 :  53.8719%
입양 확률 :  71.0483%
입양 확률 :  66.8481%
입양 확률 :  83.2895%
입양 확률 :  59.9851%
입양 확률 :  87.3223%
입양 확률 :  94.8906%
입양 확률 :  53.2306%
입양 확률 :  41.5467%
입양 확률 :  41.7502%
입양 확률 :  26.5065%
입양 확률 :  29.7671%
입양 확률 :  96.1752%
입양 확률 :  44.4502%
입양 확률 :  92.8657%
입양 확률 :  39.4909%
입양 확률 :  83.0424%
입양 확률 :  91.3181%
입양 확률 :  86.7357%
입양 확률 :  54.5117%
입양 확률 :  23.3083%
입양 확률 :  24.4934%
입양 확률 :  90.5356%
입양 확률 :  65.2899%
입양 확률 :  32.4555%
입양 확률 :  20.8052%
입양 확률 :  31.4452%
입양 확률 :  50.1362%
입양 확률 :  52.8710%
입양 확률 :  53.2796%
입양 확률 :  53.4489%
입양 확률 :  85.6347%
입양 확률 :  89.7685%
입양 확률 :  30.2853%
입양 확률 :  57.8625%
입양 확률 :  33.0917%
입양 확률 :  52.3523%
입양 확률 :  90.5433%
입양 확률 :  9